In [70]:
import pandas as pd
import numpy as np
path_sub = "./sub/"
path_build = "./build/"
from utils import *
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

from sklearn.model_selection import KFold

def fold5_mean(filename):
    gender_preds = []
    age_preds = []
    kfolder = KFold(n_splits=5, shuffle=True, random_state=2020)
    kfold = kfolder.split(users[0:3000000])
    i = 1
    gender_oof = np.zeros((3000000, 2))
    age_oof = np.zeros((3000000, 10))

    for train_index, vali_index in kfold:
        print(i,train_index, vali_index)
        age_prob = np.load(f"{path_sub}/{filename}/{i}/age_prob.npy")
        val_age_prob = np.load(f"{path_sub}/{filename}/{i}/val_age_prob.npy")
        try:
            gender_prob = np.load(f"{path_sub}/{filename}/{i}/gender_prob.npy")
            val_gender_prob = np.load(f"{path_sub}/{filename}/{i}/val_gender_prob.npy")
        except:
            print('gender not exists!!!')
            gender_prob = 1
            val_gender_prob = 1
        gender_preds.append(gender_prob)
        age_preds.append(age_prob)
        gender_oof[vali_index] = val_gender_prob
        age_oof[vali_index] = val_age_prob
        
        
        i = i+1
    gender_prob = np.array(gender_preds).mean(axis=0)
    age_prob = np.array(age_preds).mean(axis=0)
    
    age_accuracy = accuracy_score(y_true=users['age'][0:3000000]-1, y_pred=age_oof.argmax(axis=1))
    gender_accuracy = accuracy_score(y_true=users['gender'][0:3000000]-1, y_pred=gender_oof.argmax(axis=1))
    print(f'age accuracy_score:{age_accuracy}')
    print(f'gender accuracy_score:{gender_accuracy}, sum:{age_accuracy+gender_accuracy}')
    print(gender_prob.sum(), age_prob.sum())
    print('----------------------------------------------')
    return gender_prob, age_prob, gender_oof,age_oof

    
# 两个五折
gender_prob_51, age_prob_51, gender_oof_51, age_oof_51 = fold5_mean('sub-5-5fold')
#gender_prob_52, age_prob_52, gender_oof_52, age_oof_52 = fold5_mean('sub-5times-5fold')
gender_prob_53, age_prob_53, gender_oof_53, age_oof_53 = fold5_mean('sub-10-5fold')
gender_prob_54, age_prob_54, gender_oof_54, age_oof_54 = fold5_mean('sub-5unit*1-5fold')
#gender_prob_55, age_prob_55, gender_oof_55, age_oof_55 = fold5_mean('sub-5timesunit*1-5fold')
gender_prob_56, age_prob_56, gender_oof_56, age_oof_56 = fold5_mean('sub-8-5fold')


# LR
gender_prob_lr = np.load(f"{path_sub}/lr/gender_prob.npy")
age_prob_lr = np.load(f"{path_sub}/lr/age_prob.npy")
gender_val_prob_lr = np.load(f"{path_sub}/lr/val_gender_prob.npy")
age_val_prob_lr = np.load(f"{path_sub}/lr/val_age_prob.npy")

# 单模

age_prob1 = np.load(f"{path_sub}/sub-all_single_model/age_prob.npy")
gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/gender_prob.npy")
val_age_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_age_prob.npy")
val_gender_prob1 = np.load(f"{path_sub}/sub-all_single_model/val_gender_prob.npy")
print('单模：',age_prob1.sum(), gender_prob1.sum())


sub_gender_prob = (gender_prob_51 + gender_prob_53 + gender_prob_54 + gender_prob_56 + gender_prob1 + gender_prob_lr)/6
sub_age_prob = (age_prob_51 + age_prob_53 + age_prob_54 + age_prob_56 + age_prob1 + age_prob_lr)/6
sub_gender = np.argmax(sub_gender_prob ,axis=1) + 1
sub_age = np.argmax(sub_age_prob,axis=1) + 1

val_gender = np.argmax((gender_oof_51 + gender_oof_53) + gender_oof_54 + gender_oof_56 + gender_val_prob_lr ,axis=1) + 1
val_age = np.argmax((age_oof_51 + age_oof_53) + age_oof_54 + age_oof_56 + age_val_prob_lr,axis=1) + 1
print('*'*50 + '5fold' + '*'*50)
print('age accuracy_score',accuracy_score(y_true=users['age'][0:3000000], y_pred=val_age))
print('gender accuracy_score',accuracy_score(y_true=users['gender'][0:3000000], y_pred=val_gender))
print('age+gender accuracy_score',accuracy_score(y_true=users['gender'][0:3000000], y_pred=val_gender) + accuracy_score(y_true=users['age'][0:3000000], y_pred=val_age))
print('*'*50 + '5fold' + '*'*50)

val_gender1 = np.argmax(gender_oof_51[2700000:3000000] + gender_oof_53[2700000:3000000]  + gender_oof_54[2700000:3000000] + gender_oof_56[2700000:3000000] + val_gender_prob1 + gender_val_prob_lr[2700000:3000000] ,axis=1) + 1
val_age1 = np.argmax(age_oof_51[2700000:3000000] + age_oof_53[2700000:3000000] + age_oof_54[2700000:3000000] + age_oof_56[2700000:3000000] + val_age_prob1 + age_val_prob_lr[2700000:3000000],axis=1) + 1
print('*'*50 + '5fold+单模' + '*'*50)
print('age accuracy_score',accuracy_score(y_true=users['age'][2700000:3000000], y_pred=val_age1))
print('gender accuracy_score',accuracy_score(y_true=users['gender'][2700000:3000000], y_pred=val_gender1))
print('age+gender accuracy_score',accuracy_score(y_true=users['gender'][2700000:3000000], y_pred=val_gender1) + accuracy_score(y_true=users['age'][2700000:3000000], y_pred=val_age1))
print('*'*50 + '5fold+单模' + '*'*50)

# sub_gender = np.argmax((gender_prob_51 + 
#                        gender_prob_52 + 
#                        gender_prob1 + 
#                        gender_prob2)/4 ,axis=1) + 1

# sub_age = np.argmax((age_prob_51 + 
#                        age_prob_52 + 
#                        age_prob1 + 
#                        age_prob2 )/4 ,axis=1) + 1

print(sub_gender.shape, sub_age.shape)

sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = sub_age
sub['predicted_gender'] = sub_gender
print('ok！')
sub.to_csv(f"{path_sub}/submission.csv", index=False, encoding='utf-8')
np.save(f"{path_sub}/sub_gender_prob", sub_gender_prob)
np.save(f"{path_sub}/sub_age_prob", sub_age_prob)
print(sub_gender_prob.sum(), sub_age_prob.sum(), sub_age_prob.shape, sub_gender_prob.shape)
local2cos(f"{path_sub}/submission.csv", "lht-build-1259079995", "sub")
#https://lht-build-1259079995.cos.ap-guangzhou.myqcloud.com/sub/submission.csv

1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [      6       8      13 ... 2999996 2999997 2999999]
4 [      0       1       2 ... 2999997 2999998 2999999] [     10      11      16 ... 2999991 2999992 2999995]
5 [      0       2       3 ... 2999996 2999997 2999999] [      1      12      20 ... 2999976 2999983 2999998]
age accuracy_score:0.49995266666666666
gender accuracy_score:0.9480696666666667, sum:1.4480223333333333
1000000.0 999999.25
----------------------------------------------
1 [      1       6       8 ... 2999997 2999998 2999999] [      0       2       3 ... 2999990 2999993 2999994]
2 [      0       1       2 ... 2999997 2999998 2999999] [      9      24      30 ... 2999984 2999987 2999989]
3 [      0       1       2 ... 2999994 2999995 2999998] [  

In [58]:
import pandas as pd
import numpy as np
path_sub = "./sub/"
path_build = "./build/"
from utils import *
users = pd.read_pickle(f"{path_build}/train_user.pkl")
users.sort_values(by=['user_id'], ascending=[True], inplace=True)
users = users.reset_index(drop=True)

def load_model(path):
    age_prob = np.load(f"{path}/age_prob.npy")
    gender_prob = np.load(f"{path}/gender_prob.npy")

    age_val_prob = np.load(f"{path}/val_age_prob.npy")
    gender_val_prob = np.load(f"{path}/val_gender_prob.npy")
    #print(path + ":")
    #print(age_prob.shape, gender_prob.shape, age_val_prob.shape, gender_val_prob.shape)
    #print(age_prob.sum(), gender_prob.sum(), age_val_prob.sum(), gender_val_prob.sum())
    return (gender_prob, gender_val_prob, age_prob, age_val_prob)

gender_prob0, gender_val_prob0, age_prob0, age_val_prob0 = load_model(path_sub + 'sub-5')
gender_prob1, gender_val_prob1, age_prob1, age_val_prob1 = load_model(path_sub + 'sub-5times')
gender_prob2, gender_val_prob2, age_prob2, age_val_prob2 = load_model(path_sub + 'sub-10')
gender_prob3, gender_val_prob3, age_prob3, age_val_prob3 = load_model(path_sub + 'sub-5-v0')
gender_prob4, gender_val_prob4, age_prob4, age_val_prob4 = load_model(path_sub + 'sub-5-v1')
gender_prob5, gender_val_prob5, age_prob5, age_val_prob5 = load_model(path_sub + 'sub-5-v2')
gender_prob6, gender_val_prob6, age_prob6, age_val_prob6 = load_model(path_sub + 'sub-5-v3')
gender_prob7, gender_val_prob7, age_prob7, age_val_prob7 = load_model(path_sub + 'sub-7')
gender_prob8, gender_val_prob8, age_prob8, age_val_prob8 = load_model(path_sub + 'sub-5-5lstm')
gender_prob9, gender_val_prob9, age_prob9, age_val_prob9 = load_model(path_sub + 'sub-5-v4')
gender_prob10, gender_val_prob10, age_prob10, age_val_prob10 = load_model(path_sub + 'sub-5-v5')
gender_prob11, gender_val_prob11, age_prob11, age_val_prob11 = load_model(path_sub + 'sub-5-v5_1')
gender_prob12, gender_val_prob12, age_prob12, age_val_prob12 = load_model(path_sub + 'sub-5-v5_2')



#----------------------------------------------------------------
gender_prob = (gender_prob0 + 
               gender_prob5 + gender_prob7  + gender_prob9 + 
               gender_prob10 + gender_prob11 + gender_prob12) / 7
gender_val_prob = ( gender_val_prob0+
                   gender_val_prob5  + gender_val_prob7 + gender_val_prob9 + 
                   gender_val_prob10 + gender_val_prob11 + gender_val_prob12) / 7

age_prob = ( 
            age_prob5  + age_prob7 + age_prob9 + 
            age_prob10 + age_prob11 + age_prob12) / 6
age_val_prob = ( 
                age_val_prob5  + age_val_prob7 + age_val_prob9 + 
                age_val_prob10 + age_val_prob11 + age_val_prob12) / 6
#----------------------------------------------------------------

#gender_prob, gender_val_prob, age_prob, age_val_prob = load_model(path_sub)

sw = False
if sw:
    print("search wieght 。。。。。。")
    age_tune_weight = search_weight(users['age'][2700000:3000000]-1, age_val_prob, init_weight=[1.0]*10,class_num=10, step=0.001)
    print(age_tune_weight)

    gender_tune_weight = search_weight(users['gender'][2700000:3000000]-1, gender_val_prob, init_weight=[1.0]*2,class_num=2, step=0.001)
    print(gender_tune_weight)

    gender_prob_tune = np.array(gender_tune_weight)*gender_prob
    gender_pre = np.argmax(gender_prob_tune,axis=1) + 1

    age_prob_tune = np.array(age_tune_weight)*age_prob
    age_pre = np.argmax(age_prob_tune,axis=1) + 1
else:
    print(accuracy_score(y_true=users['age'][2700000:3000000]-1, y_pred=age_val_prob.argmax(axis=1)))
    print(accuracy_score(y_true=users['gender'][2700000:3000000]-1, y_pred=gender_val_prob.argmax(axis=1)))
    gender_pre = np.argmax(gender_prob,axis=1) + 1
    age_pre = np.argmax(age_prob,axis=1) + 1

0.51343
0.9493633333333333


In [59]:
np.save(f"{path_sub}/sub-all_single_model/gender_prob.npy", gender_prob)
np.save(f"{path_sub}/sub-all_single_model/age_prob.npy", age_prob)


In [60]:
np.save(f"{path_sub}/sub-all_single_model/val_gender_prob.npy", gender_val_prob)
np.save(f"{path_sub}/sub-all_single_model/val_age_prob.npy", age_val_prob)

In [39]:
sub = pd.DataFrame()
sub['user_id'] = range(3000001,4000001)
sub['predicted_age'] = age_pre
sub['predicted_gender'] = gender_pre
print('ok！')
sub.to_csv(f"{path_sub}/submission.csv", index=False, encoding='utf-8')

local2cos(f"{path_sub}/submission.csv", "lht-build-1259079995", "sub")
#https://lht-build-1259079995.cos.ap-guangzhou.myqcloud.com/sub/submission.csv

ok！


In [12]:
import os
mkdirs = ['embed','feature_series','build']
files = ['1_data_nmt.ipynb', '2_w2v.ipynb', '3_nn.ipynb', '3_nn_tf21.ipynb', 'pip_install.ipynb', 'utils.py', 'cudatoolkit-10.1.168-0.tar.bz2', 'run.py', 'run.sh']
# for mkdir in mkdirs:
#     for filename in os.listdir(mkdir):
#         print(f"{mkdir}/{filename}")
#         local2cos(f"{mkdir}/{filename}", "lht-model-1259079995", mkdir)
        
for file in files:
    print(file)
    local2cos(f"./{file}", "lht-model-1259079995", 'src')

1_data_nmt.ipynb
2_w2v.ipynb
3_nn.ipynb
3_nn_tf21.ipynb
pip_install.ipynb
utils.py
cudatoolkit-10.1.168-0.tar.bz2
run.py
run.sh


In [33]:
local2cos(f"./mnt/user_mnts.pkl", "lht-model-1259079995", 'mnt')